In [2]:
import pandas as pd

def read_data(file_path):
    print("Opening dataset: ", file_path)
    df = pd.read_csv(file_path)
    print("Dataframe shape: ", df.shape)
    print("Dataframe columns: ", df.columns, "\n")
    return df

In [3]:
#Open CSV files
true_data = read_data("Datasets/True.csv")
true_data['label'] = 1 # Add a label column to the true data

false_data = read_data("Datasets/Fake.csv")
false_data['label'] = 0 # Add a label column to the fake data

Opening dataset:  Datasets/True.csv
Dataframe shape:  (21417, 4)
Dataframe columns:  Index(['title', 'text', 'subject', 'date'], dtype='object') 

Opening dataset:  Datasets/Fake.csv
Dataframe shape:  (23481, 4)
Dataframe columns:  Index(['title', 'text', 'subject', 'date'], dtype='object') 



In [4]:
#Check if there is any null value
print("True data null values: ", true_data.isnull().sum())

#We actually don't need the date column for the classification
true_data.drop(columns=['date'])

True data null values:  title      0
text       0
subject    0
date       0
label      0
dtype: int64


,title,text,subject,label
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,1
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,1
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,1
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,1
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,1
...,...,...,...,...
21412,'Fully committed' NATO backs new U.S. approach...,BRUSSELS (Reuters) - NATO allies on Tuesday we...,worldnews,1
21413,LexisNexis withdrew two products from Chinese ...,"LONDON (Reuters) - LexisNexis, a provider of l...",worldnews,1
21414,Minsk cultural hub becomes haven from authorities,MINSK (Reuters) - In the shadow of disused Sov...,worldnews,1
21415,Vatican upbeat on possibility of Pope Francis ...,MOSCOW (Reuters) - Vatican Secretary of State ...,worldnews,1


In [5]:
#Check if there is any null value
print("False data null values: ", false_data.isnull().sum())

#The same goes for the false data
false_data.drop(columns=['date'])

False data null values:  title      0
text       0
subject    0
date       0
label      0
dtype: int64


,title,text,subject,label
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,0
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,0
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,0
...,...,...,...,...
23476,McPain: John McCain Furious That Iran Treated ...,21st Century Wire says As 21WIRE reported earl...,Middle-east,0
23477,JUSTICE? Yahoo Settles E-mail Privacy Class-ac...,21st Century Wire says It s a familiar theme. ...,Middle-east,0
23478,Sunnistan: US and Allied ‘Safe Zone’ Plan to T...,Patrick Henningsen 21st Century WireRemember ...,Middle-east,0
23479,How to Blow $700 Million: Al Jazeera America F...,21st Century Wire says Al Jazeera America will...,Middle-east,0


In [27]:
#Merge the two datasets
dataset = pd.concat([true_data, false_data])

In [30]:
#Importing CountVectorizer to implement Bag of Words
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer()

In [31]:
#test train data split
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(dataset.text,dataset.label,test_size=.2,random_state=1)

In [32]:
#Fitting the classifier to the training data
cv_1=cv.fit_transform(X_train)

In [33]:
#Using Random Forest to predict test values
from sklearn.ensemble import RandomForestClassifier
rf=RandomForestClassifier()
rf.fit(cv_1,y_train)
y_pred_randomForest=rf.predict(cv.transform(X_test))

In [34]:
#Computing the accuracy of the model
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test,y_pred_randomForest))

0.9907572383073496


In [84]:
#Plotting the result
import matplotlib.pyplot as plt
import numpy as np

ConnectionAbortedError
def plot_dataset(X,Y):
    colors = ['r', 'b', 'g', 'k', 'c', 'm', 'y']

    n_classes = np.unique(Y).size

    for k in range(n_classes):
        xk = X[Y==k, :] 
        plt.scatter(xk[:, 0], xk[:, 1], c=colors[k], label='class ' + str(k))

    plt.legend()
    plt.xlabel('x0')
    plt.ylabel('x1')
    plt.title('Gaussian data')

#plot_dataset(X_train, y_train)
 

     mean radius  mean texture  mean perimeter  mean area  mean smoothness  \
293       11.850         17.46           75.54      432.7          0.08372   
332       11.220         19.86           71.94      387.3          0.10540   
565       20.130         28.25          131.20     1261.0          0.09780   
278       13.590         17.84           86.24      572.3          0.07948   
489       16.690         20.20          107.10      857.6          0.07497   
..           ...           ...             ...        ...              ...   
277       18.810         19.98          120.90     1102.0          0.08923   
9         12.460         24.04           83.97      475.9          0.11860   
359        9.436         18.32           59.82      278.6          0.10090   
192        9.720         18.22           60.73      288.1          0.06950   
559       11.510         23.93           74.52      403.5          0.09261   

     mean compactness  mean concavity  mean concave points  mea